In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd 'drive/MyDrive/'

#Imports

In [6]:
!pip3 install git+https://github.com/nnaisense/pgpelib.git#egg=pgpelib

from IPython.display import clear_output
from pgpelib import PGPE
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
from tensorflow.keras.utils import get_file
import numpy as np
import matplotlib.pyplot as plt
import os
import io
import requests
import random
import glob
import pickle
import time

%matplotlib inline
clear_output()

#Polygon renderer

In [1]:
class TrianglesPainter:
    def __init__(self, h, w, n_triangles=10, alpha_scale=0.1, coordinate_scale=1.0):
        self.h = h
        self.w = w
        self.n_triangles = n_triangles
        self.alpha_scale = alpha_scale
        self.coordinate_scale = coordinate_scale
        
    def render(self, params):
        """
        render the parameters on a empty
        blank canvas and return the image
        as an array
        """
        h, w = self.h, self.w
        alpha_scale = self.alpha_scale
        coordinate_scale = self.coordinate_scale
        
        params = params.reshape(-1, 10).copy()
        n_triangles, n_features = params.shape[0], params.shape[1]

        # normalize features 
        for j in range(n_features):
            params[:, j] = (params[:, j] - params[:, j].min()) / (params[:, j].max() - params[:, j].min())

        # initialize blank canvas for image to be rendered on
        img = Image.new("RGB", (w, h), (0, 0, 0))
        draw = ImageDraw.Draw(img, 'RGBA')
        
        params = params.tolist()

        # draw the polygons according to the parameters 
        for i in range(n_triangles):
            slice = params[i]
            
            x0, y0, x1, y1, x2, y2, r, g, b, a = slice
            xc, yc = (x0 + x1 + x2) / 3. , (y0 + y1 + y2) / 3.
            
            x0, y0 = xc + (x0 - xc) * coordinate_scale, yc + (y0 - yc) * coordinate_scale
            x1, y1 = xc + (x1 - xc) * coordinate_scale, yc + (y1 - yc) * coordinate_scale
            x2, y2 = xc + (x2 - xc) * coordinate_scale, yc + (y2 - yc) * coordinate_scale
            
            x0, x1, x2 = int(x0 * h), int(x1 * h), int(x2 * h)
            y0, y1, y2 = int(y0 * w), int(y1 * w), int(y2 * w)
            r, g, b, a = int(r * 255), int(g * 255), int(b * 255), int(a * alpha_scale * 255)
            
            draw.polygon([(y0, x0), (y1, x1), (y2, x2)], (r, g, b, a))
        
        del draw
        
        img_arr = np.array(img)

        return img_arr

# Utility functions

In [2]:
def img2arr(img):
    return np.array(img)

def arr2img(arr):
    return Image.fromarray(arr)

def convert2rgb(image):
    h, w = image.size
    img = Image.new('RGB', (h, w))
    img.paste(image)
    return img 

def resize(img, size):
    img = img.resize(size, Image.LANCZOS)
    img_arr = img2arr(img)
    return img_arr

def get_target_shape(image):
    return image.shape[0], image.shape[1]

# get random fake person from https://thispersondoesnotexist.com
def get_image(image_url):
    r = requests.get(image_url, headers={'User-Agent': 'My User Agent 1.0'}).content
    image = Image.open(io.BytesIO(r))
    image = convert2rgb(image)
    image = resize(image, (256,256))
    return image

def display_image(arr, gen):
    fig = plt.figure()
    fig.suptitle(f"Solution at generation: {gen}")
    plt.axis('off')
    plt.imshow(arr)
    plt.show()

def save_loss(data, filename):
    np.save(f'loss_{filename}', np.asarray(data), allow_pickle=True)
    fig = plt.figure()
    fig.suptitle("Loss")
    plt.plot(data)
    plt.show()
    fig.savefig(f"loss_{filename}.jpg")
  
def make_gif(images_dir):
    frames = [Image.open(image) for image in glob.glob(f"{images_dir}/*.jpg")]
    frame_one = frames[0]
    frame_one.save("output", format="GIF", append_images=frames,
               save_all=True, duration=500, loop=0)
    
def show_result(images_dir, n):
    image = img2arr((Image.open(images_dir + os.listdir(images_dir)[-2])))
    display_image(image, n)

#Fitness function

In [3]:
def fitness_fn(params, painter, target, loss_fn):
    '''
    fitness function can use either the l1/l2 pixelwise loss (default: l2) to evaluate the generated solutions.
    returns the average negative loss as pgpe is a maximizing function
    '''

    array = painter.render(params).astype(np.float32) / 255.
    target = target.astype(np.float32) / 255.

    if loss_fn == 'l1':
        loss = np.abs(array - target)
    else:
        loss = (array - target)**2
    return -np.mean(loss) 

#Main

In [10]:
#@title Settings
def run():
    # painter parameters
    n_triangles = 300 #@param {type : "number"}
    alpha_scale =  0.1# @param {type: "number"}
    coordinate_scale = 1.0 #@param {type : "number"}

    # hyperparameters
    n_iterations = 3000 #@param {type : "integer"}
    n_params = n_triangles * 10
    n_population = 128 #@param [64, 128, 256, 512] {type : "raw"}
    loss_type = 'l2' #@param ['l1', 'l2'] {type : "string"}
    optimizer = 'adam' #@param ['adam', 'clipup'] {type : "string"}

    target = get_image("https://thispersondoesnotexist.com/image")
    height, width = get_target_shape(target)
    
    painter = TrianglesPainter(
        h=height, 
        w=width,
        n_triangles=n_triangles,
        alpha_scale=alpha_scale,
        coordinate_scale=coordinate_scale
    )

    '''
    PGPE is an optimization algorithm for computing approximate policy gradients 
    reference: https://github.com/nnaisense/pgpelib
    '''
    solver = PGPE(
        solution_length = n_params, 
        popsize = n_population,
        optimizer=optimizer
    )
 
    ROOT = os.getcwd()
    PATH = f'config_{optimizer}_{n_iterations}_{n_params}_{n_population}'
    OUTPUT_DIR = ROOT + '/images/' + PATH
   
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
    
    fitness_loss = []
   
    # show target image
    target_image = Image.fromarray(target)
    target_image.save(f'{OUTPUT_DIR}/target.jpg')
    fig = plt.figure()
    fig.suptitle(f'Target')
    plt.axis('off')
    plt.imshow(np.array(target_image))
    plt.show()

    for i in tqdm(range(n_iterations)):
        
        # we ask the solver to generate solutions and run the fitness function to update their parameters
        solutions = solver.ask()
        fitnesses = [fitness_fn(solutions[i], painter, target, loss_fn=loss_type) for i in range(len(solutions))]
        solver.tell(fitnesses)
        fitness_loss.append(-np.mean(fitnesses))

        # render and save the best solution found so far
        arr = painter.render(solver.center) 
        img = Image.fromarray(arr)
        img.save(f'{OUTPUT_DIR}/sol_{i}.jpg')

        if i % 10 == 0:
            display_image(arr, i)
           
    clear_output()

    save_loss(fitness_loss, filename=PATH) 
    show_result(OUTPUT_DIR + '/', n_iterations)

In [ ]:
run()